In [1]:
import pandas as pd
import numpy as np
from os import listdir
import keras
# import cv2
from skimage.io import imread
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
train_path = "train/"
test_path = "test/"
label_path = './train.csv'
train_files = listdir(train_path)
test_files = listdir(test_path)

In [3]:
train_labels = pd.read_csv(label_path)
train_labels.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [4]:
names_dict = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}
# reverse_names_dict = dict((v,k) for k,v in names_dict.items())

In [5]:
def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = names_dict[int(num)]
        row.loc[name] = 1
    return row

for key in names_dict.keys():
    train_labels[names_dict[key]] = 0
train_labels = train_labels.apply(fill_targets, axis=1)
train_labels["number_of_targets"] = train_labels.drop(["Id", "Target"],axis=1).sum(axis=1)

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
for train_idx, test_idx in kf.split(train_labels.index.values):
    partition = {}
    partition["train"] = train_labels.Id.values[train_idx]
    partition["validation"] = train_labels.Id.values[test_idx]
#     X_train, X_test = train_labels.Id.values[train_idx], train_labels.Id.values[test_idx]
#     y_train, y_test = train_labels.Id.values[train_idx], train_labels.Id.values[test_idx]

In [7]:
partition

{'train': array(['00070df0-bbc3-11e8-b2bc-ac1f6b6435d0',
        '000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0',
        '000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', ...,
        'ab33dff8-bba7-11e8-b2ba-ac1f6b6435d0',
        'ab351f1c-bbb6-11e8-b2ba-ac1f6b6435d0',
        'ab385b2e-bbba-11e8-b2ba-ac1f6b6435d0'], dtype=object),
 'validation': array(['ab3978aa-bbbb-11e8-b2ba-ac1f6b6435d0',
        'ab3b9258-bbab-11e8-b2ba-ac1f6b6435d0',
        'ab3ca16e-bbc1-11e8-b2bb-ac1f6b6435d0', ...,
        'fff189d8-bbab-11e8-b2ba-ac1f6b6435d0',
        'fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0',
        'fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0'], dtype=object)}

In [8]:
class ModelParameters(object):
    path = train_path
    num_classes=28
    image_rows=512
    image_cols=512
    batch_size=100
    n_channels=3
    shuffle=False
    scaled_row_dim = 139
    scaled_col_dim = 139 
    n_epochs=100

parameter = ModelParameters()

In [9]:
class ImagePreprocessor:
    
    def __init__(self, modelparameter):
        self.parameter = modelparameter
        self.path = self.parameter.path
        self.scaled_row_dim = self.parameter.scaled_row_dim
        self.scaled_col_dim = self.parameter.scaled_col_dim
        self.n_channels = self.parameter.n_channels
    
    def preprocess(self, image):
        image = self.resize(image)
        image = self.reshape(image)
        image = self.normalize(image)
        return image
    
    def resize(self, image):
        return resize(image, (self.scaled_row_dim, self.scaled_col_dim))
    
    def reshape(self, image):
        return np.reshape(image, (image.shape[0], image.shape[1], self.n_channels))
    
    def normalize(self, image):
#         image /= 255
#         return image
        return (image / 255.0 - 0.5) / 0.5
            
    def load_image(self, image_id):
        image = np.zeros(shape=(512,512,4))
        image[:,:,0] = imread(self.basepath + image_id + "_green" + ".png")
        image[:,:,1] = imread(self.basepath + image_id + "_blue" + ".png")
        image[:,:,2] = imread(self.basepath + image_id + "_red" + ".png")
        image[:,:,3] = imread(self.basepath + image_id + "_yellow" + ".png")
        return image[:,:,0:self.parameter.n_channels]

In [10]:
preprocessor = ImagePreprocessor(parameter)

In [11]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor):
        self.params = modelparameter
        self.labels = labels
        self.list_IDs = list_IDs
        self.dim = (self.params.scaled_row_dim, self.params.scaled_col_dim)
        self.batch_size = self.params.batch_size
        self.n_channels = self.params.n_channels
        self.num_classes = self.params.num_classes
        self.preprocessor = imagepreprocessor
        self.shuffle = self.params.shuffle
        self.on_epoch_end()
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def get_targets_per_image(self, identifier):
        return self.labels.loc[self.labels.Id==identifier].drop(
                ["Id", "Target", "number_of_targets"], axis=1).values
            
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        for i, identifier in enumerate(list_IDs_temp):
            image = self.preprocessor.load_image(identifier)
            image = self.preprocessor.preprocess(image)
            X[i] = image
            y[i] = self.get_targets_per_image(identifier)
        return X, y
    
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
 
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

In [12]:
df = pd.read_csv('sample_submission.csv')
partition2 = np.array(df['Id'])

In [13]:
class PredictGenerator:
    
    def __init__(self, predict_Ids, imagepreprocessor, predict_path):
        self.preprocessor = imagepreprocessor
        self.preprocessor.basepath = predict_path
        self.identifiers = predict_Ids
    
    def predict(self, model):
        y = np.empty(shape=(len(self.identifiers), self.preprocessor.parameter.num_classes))
        for n in range(len(self.identifiers)):
            image = self.preprocessor.load_image(self.identifiers[n])
            image = self.preprocessor.preprocess(image)
            image = image.reshape((1, *image.shape))
            y[n] = model.predict(image)
        return y

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import binary_crossentropy
from keras.optimizers import Adadelta
from keras.models import load_model

class BaseLineModel:
    
    def __init__(self, modelparameter):
        self.params = modelparameter
        self.num_classes = self.params.num_classes
        self.img_rows = self.params.scaled_row_dim
        self.img_cols = self.params.scaled_col_dim
        self.n_channels = self.params.n_channels
        self.input_shape = (self.img_rows, self.img_cols, self.n_channels)
        self.my_metrics = ['accuracy']
    
    def build_model(self):
        self.model = Sequential()
        self.model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=self.input_shape))
        self.model.add(Conv2D(32, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(self.num_classes, activation='sigmoid'))

    def compile_model(self):
        self.model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=self.my_metrics)
    
    def set_generators(self, train_generator, validation_generator):
        self.training_generator = train_generator
        self.validation_generator = validation_generator
    
    def learn(self):
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.params.n_epochs, 
                    steps_per_epoch=100,
                    use_multiprocessing=True,
                    validation_steps=50,
                    workers=8)
    
    def score(self):
        return self.model.evaluate_generator(generator=self.validation_generator,
                                      use_multiprocessing=True, 
                                      workers=8)

    def predict(self, predict_generator):
        y = predict_generator.predict(self.model)
        return y
    
    def save(self, modeloutputpath):
        self.model.save(modeloutputpath)
    
    def load(self, modelinputpath):
        self.model = load_model(modelinputpath)

In [15]:
labels = train_labels
print("Number of samples in train: {}".format(len(partition["train"])))
print("Number of samples in validation: {}".format(len(partition["validation"])))

Number of samples in train: 20715
Number of samples in validation: 10357


In [17]:
test_path = "test/"
train_path = "train/"
training_generator = DataGenerator(partition['train'], labels, parameter, preprocessor)
validation_generator = DataGenerator(partition['validation'], labels, parameter, preprocessor)
predict_generator = PredictGenerator(partition['validation'], preprocessor, train_path)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model = BaseLineModel(parameter)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
history = model.learn()
#model.save("baseline_model.h5")
proba_predictions = model.predict(predict_generator)
baseline_proba_predictions = pd.DataFrame(proba_predictions, columns=train_labels.drop(
    ["Target", "number_of_targets", "Id"], axis=1).columns)
baseline_proba_predictions.to_csv("baseline_predictions.csv")

In [ ]:
baseline_proba_predictions

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9430922614536483107
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11273211085
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 3748309959796633748
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:03:00.0, compute capability: 3.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 11273211085
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 14563035596431922012
physical_device_desc: "device: 1, name: Tesla K40m, pci bus id: 0000:04:00.0, compute capability: 3.5"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 11273211085
locality {
  bus_id: 2
  numa_node: 1
  links {
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 11862455148348870157
physical_dev

In [18]:
import keras.backend as K

def base_f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return f1

def f1_min(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.min(f1)

def f1_max(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.max(f1)

def f1_mean(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.mean(f1)

def f1_std(y_true, y_pred):
    f1 = base_f1(y_true, y_pred)
    return K.std(f1)

In [19]:
class TrackHistory(keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [20]:
class ImprovedDataGenerator(DataGenerator):
    
    # in contrast to the base DataGenerator we add a target wishlist to init
    def __init__(self, list_IDs, labels, modelparameter, imagepreprocessor, target_wishlist):
        super().__init__(list_IDs, labels, modelparameter, imagepreprocessor)
        self.target_wishlist = target_wishlist
    
    def get_targets_per_image(self, identifier):
        return self.labels.loc[self.labels.Id==identifier][self.target_wishlist].values


In [22]:
use_dropout = False
from keras.models import Model, Sequential,load_model
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.losses import binary_crossentropy
from inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import Adadelta,Adam
from keras.models import load_model
import tensorflow as tf
from keras.layers import Input

class ImprovedModel(BaseLineModel):
    
    def __init__(self, modelparameter,
                 use_dropout,
                 my_metrics=[f1_mean, f1_std, f1_min, f1_max]):
        
        super().__init__(modelparameter)
        self.my_metrics = my_metrics
        self.use_dropout = use_dropout
        
    def learn(self):
        self.history = TrackHistory()
        return self.model.fit_generator(generator=self.training_generator,
                    validation_data=self.validation_generator,
                    epochs=self.params.n_epochs, 
                    use_multiprocessing=True,
                    workers=8,
                    steps_per_epoch=100,
                    validation_steps=50,                                     
                    callbacks = [self.history])
    
    def build_model(self):
        #input_tensor = Input(shape=self.input_shape)
        pretrain_model = InceptionResNetV2(
            include_top=True, 
            #weights='imagenet', 
            weights = None,
            input_shape=self.input_shape,
            #input_tensor = input_tensor
            classes = 28
            ) 
        
        self.model = pretrain_model
        
#         self.model = Sequential()
# #         for l in pretrain_model.layers:
# #             self.model.add(l)            
#         self.model.add(pretrain_model)
#         self.model.add(Conv2D(128, kernel_size=(1,1), activation='relu'))
#         self.model.add(Flatten())
#         if self.use_dropout:
#             self.model.add(Dropout(0.25))
#         self.model,add(Dense(512, activation='relu'))
#         if self.use_dropout:
#             self.model.add(Dropout(0.25))
#         self.model.add(Dense(self.num_classes, activation='sigmoid'))         
    

In [23]:
print(keras.__version__)

2.0.8


In [22]:
parameter = ModelParameters()
preprocessor = ImagePreprocessor(parameter)
labels = train_labels
training_generator = DataGenerator(partition['train'], labels,
                                           parameter, preprocessor)
validation_generator = DataGenerator(partition['validation'], labels,
                                             parameter, preprocessor)
predict_generator = PredictGenerator(partition2, preprocessor, test_path)

In [23]:
config = tf.ConfigProto(intra_op_parallelism_threads=4,\
        inter_op_parallelism_threads=4, allow_soft_placement=True,\
        device_count = {'CPU' : 4, 'GPU' : 1})
session = tf.Session(config=config)
K.set_session(session)

In [24]:
import warnings
warnings.filterwarnings("ignore")
model = ImprovedModel(parameter, use_dropout=use_dropout)
model.build_model()
model.compile_model()
model.set_generators(training_generator, validation_generator)
epoch_history = model.learn()
proba_predictions = model.predict(predict_generator)
#model.save("improved_model.h5")
improved_proba_predictions = pd.DataFrame(proba_predictions)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
100/100 [==============================] - 399s - loss: 0.1810 - f1_mean: 0.0155 - f1_std: 0.0623 - f1_min: 0.0000e+00 - f1_max: 0.3064 - val_loss: 0.1878 - val_f1_mean: 0.0000e+00 - val_f1_std: 0.0000e+00 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0000e+00
Epoch 2/100
100/100 [==============================] - 299s - loss: 0.1705 - f1_mean: 0.0195 - f1_std: 0.0758 - f1_min: 0.0000e+00 - f1_max: 0.3706 - val_loss: 0.1922 - val_f1_mean: 5.9281e-04 - val_f1_std: 0.0031 - val_f1_min: 0.0000e+00 - val_f1_max: 0.0166
Epoch 3/100
100/100 [==============================] - 300s - loss: 0.1639 - f1_mean: 0.0271 - f1_std: 0.0930 - f1_min: 0.0000e+00 - f1_max: 0.4392 - val_loss: 0.1825 - val_f1_mean

100/100 [==============================] - 300s - loss: 0.0447 - f1_mean: 0.5144 - f1_std: 0.3913 - f1_min: 0.0000e+00 - f1_max: 0.9993 - val_loss: 0.2232 - val_f1_mean: 0.1315 - val_f1_std: 0.2230 - val_f1_min: 0.0000e+00 - val_f1_max: 0.7992
Epoch 59/100
100/100 [==============================] - 299s - loss: 0.0451 - f1_mean: 0.5155 - f1_std: 0.3921 - f1_min: 0.0000e+00 - f1_max: 0.9975 - val_loss: 0.2113 - val_f1_mean: 0.1010 - val_f1_std: 0.1883 - val_f1_min: 0.0000e+00 - val_f1_max: 0.6943
Epoch 60/100
100/100 [==============================] - 300s - loss: 0.0451 - f1_mean: 0.5150 - f1_std: 0.3906 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.1926 - val_f1_mean: 0.1200 - val_f1_std: 0.2039 - val_f1_min: 0.0000e+00 - val_f1_max: 0.6730
Epoch 61/100
100/100 [==============================] - 299s - loss: 0.0450 - f1_mean: 0.5072 - f1_std: 0.3918 - f1_min: 0.0000e+00 - f1_max: 1.0000 - val_loss: 0.2258 - val_f1_mean: 0.0913 - val_f1_std: 0.1684 - val_f1_min: 0.0000e+00 - val_

In [25]:
# pd.DataFrame(proba_predictions)
model.save("InceptionResNetv2_model.h5")

In [26]:
improved_proba_predictions.to_csv("InceptionResNetV2_predictions.csv")